In [2]:
SELECT * FROM dy1d_factor_ret_cne6_zx20 LIMIT 100

SyntaxError: invalid syntax (313755162.py, line 1)

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm import tqdm

host="192.168.1.111"
user="reader"
password="Reader_TT1"
database="datayesdb"

# Sample query
sql_query = """
    SELECT
        a.PARTY_ID,
        a.TICKER_SYMBOL,
        a.PUBLISH_DATE,
        a.END_DATE,
        a.CURRENCY_CD,
        a.TOTAL_MF,
        a.CASH_ON_HAND,
        a.BANK_DEP,
        a.OTH_MF,
        a.SPEC_MF,
        a.ADJUSTED_MF,
        a.TOT_DEP_ABROAD,
        a.TOT_RES_FD /*例举其中几个指标，其他字段名称见数据字典*/
    FROM
    fdmt_mf_item a
    JOIN md_security b ON a.PARTY_ID = b.PARTY_ID /*关联证券主表获取证券简称*/
    AND b.EXCHANGE_CD IN ('XSHG', 'XSHE')
    AND b.ASSET_CLASS = 'E' /*限制为沪深股票*/
    WHERE
    b.SEC_SHORT_NAME LIKE '%京山轻机%' /*输入公司名称，支持模糊查询*/
    AND a.END_DATE >= '2018-01-01'
    ORDER BY
    a.END_DATE DESC,
    a.PUBLISH_DATE DESC
"""

sql_query = """
    SELECT 
        * 
    FROM 
        dy1d_factor_ret_cne6_zx20
    WHERE
        TRADE_DATE >= '20010102'
        ORDER BY
        TRADE_DATE
"""

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
df = pd.DataFrame(engine.connect().execute(text(sql_query))).drop(columns=["UPDATE_TIME", "ID"]).rename(columns={"TRADE_DATE": "日期"}).set_index("日期").astype(float)
assets = df.columns.to_list()
df = (1.0 + df).cumprod().reset_index()
df.fillna(-1, inplace=True)
for asset in tqdm(assets):
    tmp = df[["日期", asset]].rename(columns={asset: "收盘价(元)"})
    tmp["日期"] = tmp["日期"].astype(str)
    tmp.to_csv(f"{asset}.csv")


100%|██████████| 54/54 [00:00<00:00, 188.45it/s]
